In [1]:
import os
import pandas as pd
import torch

from P2_utils.data_pro import *
from P2_utils.Mo import *
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("husl") #设置所有图的颜色，使用hls色彩空间

%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:

# 基础设置
data_list = ['10cm湿度(kg/m2)', '40cm湿度(kg/m2)',
             '100cm湿度(kg/m2)', '200cm湿度(kg/m2)',
             '土壤蒸发量(W/m2)', '土壤蒸发量(mm)',
             '降水量(mm)']
scaler = MinMaxScaler(feature_range=(0, 1))
# 参数设置
seq_length=1 # 时间步长
# input_size=6
input_size = len(data_list)
num_layers=2
hidden_size=6
batch_size=2
n_iters = 16000
lr=0.001
# output_size=6
output_size = len(data_list)

model_save_path = 'P2_2_model_save/'
picture_save_path = 'P2_3_lstm_figure/'
csv_save_path = 'P2_3_data/'

In [5]:
# 获取数据
for month_idx in range(1, 13):
    print("第 %d 个月正开始" % month_idx)
    data_path = 'P2_1_data/month_data_%d.csv' % month_idx
    # print(data_path)
    origin_data = pd.read_csv(data_path)
    years = origin_data['年份'].values
    # print(years[-1])
    years = np.append(years, years[-1] + 1)
    # print(years)
    years_dict = {"年份": years}
    temp_data = origin_data[data_list]
    temp_data.astype('float32')
    dataset_month = scaler.fit_transform(temp_data)
    x_data = fit_lstm_model_future(dataset_month, input_size=input_size)

    # 定义模型
    model_name = os.path.join(model_save_path, 'month_data_%d_final.h5'%month_idx)
    model = Net(input_size,hidden_size,num_layers,output_size,batch_size,seq_length)
    model.load_state_dict(torch.load(model_name))
    # print(model)

    y_future = model(x_data)
    y_future_numpy = scaler.inverse_transform(y_future.detach().numpy())
    # print(y_future_numpy)

    y_future_dataframe = pd.DataFrame(y_future_numpy[-1].reshape(1, len(data_list)))
    y_future_dataframe.columns = data_list
    y_future_dataframe = pd.concat([temp_data, y_future_dataframe], axis=0)
    y_future_dataframe = y_future_dataframe.reset_index(drop=True)
    # print(y_future_dataframe)
    years_dict_dataframe = pd.DataFrame(years_dict)
    # print(years_dict_dataframe)
    y_future_dataframe_new = pd.concat([years_dict_dataframe, y_future_dataframe], axis=1)

    print(y_future_dataframe_new)
    future_csv_name = os.path.join(csv_save_path, 'month_data_%d_future.csv' % month_idx)
    y_future_dataframe_new.to_csv(future_csv_name)
    # break


第 1 个月正开始
        年份  10cm湿度(kg/m2)  40cm湿度(kg/m2)  100cm湿度(kg/m2)  200cm湿度(kg/m2)  \
0   2012.0      13.730000      30.870000       42.790000      168.560000   
1   2013.0      14.090000      42.710000       55.020000      168.320000   
2   2014.0      10.550000      35.220000       52.760000      167.960000   
3   2015.0      11.170000      35.810000       44.520000      167.720000   
4   2016.0      14.110000      44.440000       47.890000      167.530000   
5   2017.0      12.480000      46.760000       45.860000      167.370000   
6   2018.0      10.690000      34.050000       42.650000      167.280000   
7   2019.0      10.270000      46.070000       63.680000      167.030000   
8   2020.0      12.210000      33.850000       47.070000      166.700000   
9   2021.0      15.170000      50.400000       82.440000      165.920000   
10  2022.0      12.450000      52.140000       93.450000      164.480000   
11  2023.0      10.262173      34.319679       52.550564      167.885895   

 

In [6]:


# 获取数据
for month_idx in range(4, 13):
    print("第 %d 个月正开始" % month_idx)
    data_path = 'P2_3_data/month_data_%d_future.csv' % month_idx
    # print(data_path)
    origin_data = pd.read_csv(data_path)
    years = origin_data['年份'].values
    # print(years[-1])
    years = np.append(years, years[-1] + 1)
    # print(years)
    years_dict = {"年份": years}
    temp_data = origin_data[data_list]
    temp_data.astype('float32')
    dataset_month = scaler.fit_transform(temp_data)
    x_data = fit_lstm_model_future(dataset_month, input_size=input_size)

    # 定义模型
    model_name = os.path.join(model_save_path, 'month_data_%d_final.h5'%month_idx)
    model = Net(input_size,hidden_size,num_layers,output_size,batch_size,seq_length)
    model.load_state_dict(torch.load(model_name))
    # print(model)

    y_future = model(x_data)
    y_future_numpy = scaler.inverse_transform(y_future.detach().numpy())
    # print(y_future_numpy)


    # break

第 4 个月正开始
        年份  10cm湿度(kg/m2)  40cm湿度(kg/m2)  100cm湿度(kg/m2)  200cm湿度(kg/m2)  \
0   2012.0      11.970000      30.730000       42.760000      168.560000   
1   2013.0      11.820000      42.600000       54.960000      168.320000   
2   2014.0      15.500000      42.330000       52.560000      167.960000   
3   2015.0      15.310000      35.630000       44.480000      167.720000   
4   2016.0      11.050000      44.070000       47.820000      167.530000   
5   2017.0      11.060000      46.280000       45.810000      167.370000   
6   2018.0      14.550000      33.840000       42.620000      167.280000   
7   2019.0      11.220000      45.670000       63.410000      167.030000   
8   2020.0      13.730000      33.650000       46.990000      166.700000   
9   2021.0      14.180000      49.840000       81.940000      165.920000   
10  2022.0      15.707405      42.427143       52.451107      167.992188   
11  2023.0      11.777665      42.653835       55.009293      168.340591   

 

In [5]:
# 验证


# 获取数据
for month_idx in range(1, 13):
    print("第 %d 个月正开始" % month_idx)
    data_path = 'P2_1_data/month_data_%d.csv' % month_idx
    # print(data_path)
    origin_data = pd.read_csv(data_path)
    years = origin_data['年份'].values
    # print(years[-1])
    years = np.append(years, years[-1] + 1)
    # print(years)
    years_dict = {"年份": years}
    temp_data = origin_data[data_list]
    temp_data.astype('float32')
    dataset_month = scaler.fit_transform(temp_data)
    x_data, y_data = fit_lstm_model(dataset_month, input_size=input_size, is_future=True)

    # 定义模型
    model_name = os.path.join(model_save_path, 'month_data_%d_final.h5'%month_idx)
    model = Net(input_size,hidden_size,num_layers,output_size,batch_size,seq_length)
    model.load_state_dict(torch.load(model_name))
    # print(model)

    y_future = model(x_data)
    y_future_numpy = scaler.inverse_transform(y_future.detach().numpy())
    y_ground_numpy = scaler.inverse_transform(y_data.detach().numpy())
    print(score(y_ground_numpy, y_future_numpy[:-1]))
    # print(y_future_numpy)


    # break


第 1 个月正开始
(97.218315, 25.880648, 0.23822165551427954)
第 2 个月正开始
(82.72149, 23.931555, 0.2986246966015876)
第 3 个月正开始
(23.396246, 9.482178, 1.2653995933622488)
第 4 个月正开始
(7.121184, 4.853948, 0.5882555968533716)
第 5 个月正开始
(10.315517, 5.9930396, 0.6551752262048908)
第 6 个月正开始
(14.8417015, 8.286005, 1.0921234941244384)
第 7 个月正开始
(16.035559, 7.553169, 0.07265784053023751)
第 8 个月正开始
(14.701346, 8.227949, 0.8758551908404923)
第 9 个月正开始
(13.492939, 7.5443015, 0.8691012372332232)
第 10 个月正开始
(11.94745, 6.305768, 0.6763333014111375)
第 11 个月正开始
(78.14999, 22.046911, 0.6925240748741571)
第 12 个月正开始
(77.495384, 21.701153, 0.5600445006408655)
